<a href="https://colab.research.google.com/github/harshitadd/AI4BharatTranslation/blob/main/LaBSEAligner_SelfVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers
!pip install scipy
!pip install sentence-splitter
!pip install indic-nlp-library

In [8]:
import sys
from indicnlp.tokenize import indic_tokenize
from indicnlp.tokenize import sentence_tokenize 
from sentence_transformers import SentenceTransformer
import numpy as np
import re 
import csv
from sentence_splitter import SentenceSplitter, split_text_into_sentences
from scipy.spatial import distance

In [3]:
model = SentenceTransformer('LaBSE')

100%|██████████| 1.75G/1.75G [01:01<00:00, 28.3MB/s]


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
def readfromsource(en,ta):
  try:
    with open(en,'r') as file: 
      content = file.read()
      patterns= ['-+','%',' \* ','\d{1}\.\d{1}','#','\d{1}','(\d{2}#)', '(\d{3}#)', '(\d{2}\.)', '(\d{3}\.)']
      for pattern in patterns:
        content = re.sub(pattern,'',content)
      content = content.replace('\n',' ')
      splitter = SentenceSplitter(language='en')
      so_sentences = splitter.split(content)
      print('English Sentence Count for ' + str(i) + ' Document is '+ str(len(so_sentences)))
      file.close()
      
    with open(ta,'r') as file:
      content = file.read()
      patterns = ['!','}','{','\*','~','@','/','!  ','=','>>','“;','—',',','&',' \| ','\(\)','\)','\(  \)','\(','-+','%','[A-Z]+','[a-z]+',' \* ','#','\d{1}','(\d{2}\.)','(\d{4}\.)','(\d{3}\.)','(\d{1}\.)','(\d{2}#)', '(\d{3}#)','\d{4}','\d{3}','\d{2}','\d{4,}','\[','\]','\[\]' ] #CHANGE THIS IF THE LANGUAGE USES . as delimeter - For languages which have Poorna Virama as delimet
      for pattern in patterns:
        content = re.sub(pattern,' ',content)
      content = content.replace('\n', '  ')
      content = content.replace('|','।')
      content = re.sub('( \.)',' ',content)
      content = re.sub('( . . .)+',' ',content)
      ta_sentences = sentence_tokenize.sentence_split(content, lang='ta') 
      print('Tamil Sentence Count for ' + str(i) + ' Document is '+ str(len(ta_sentences)))
      file.close()   
    return so_sentences, ta_sentences 
  except: 
    return [], [] 


In [35]:
def LaBSEembeddings(source, target):
  '''
  Generate LABSE embeddings
  Note: Inputs are array of strings
  '''           
  embeddings_input_1 = model.encode(source,show_progress_bar=True)
  embeddings_input_2 = model.encode(target,show_progress_bar=True)    
  print("LABSE embedding generation finished")
  return embeddings_input_1, embeddings_input_2
  

In [36]:
def match_target_sentence(source_embedding, target_embeddings):
  '''
  Calculate cosine similarity using scipy distance method
  '''
  distances = distance.cdist(source_embedding, target_embeddings, "cosine")[0]
  min_index = np.argmin(distances)
  min_distance = 1 - distances[min_index]
  if min_distance >= 0.7:
      return min_index, min_distance, "MATCH"
  else:
      return min_index, min_distance, "NOMATCH"     

In [37]:
def align(source_embeddings, target_embeddings):
  target_map = {}
  source_map = {}
  for s in range(len(source_embeddings)):
    source_map[s] = so_sentences[s]
  for t in range(len(target_embeddings)):
    target_map[t] = ta_sentences[t]
  aligned_phrases= {}
  for s in range(len(source_embeddings)) :
      alignments = match_target_sentence([source_embeddings[s]], target_embeddings)
      if alignments is not None and alignments[2] is "MATCH":
        l = []
        l.append(target_map[alignments[0]])
        l.append(alignments[1])
        aligned_phrases[source_map[s]] = l
  return aligned_phrases   

In [38]:
final = 0 
with open('matched.csv', 'w') as matching_file:
  for i in range(0,17):
    en = '/content/drive/MyDrive/eng_0_' + str(i) + '.txt'
    ta = '/content/drive/MyDrive/tam_0_' + str(i) + '.txt'
    so_sentences, ta_sentences = readfromsource(en,ta)
    source_embeddings, target_embeddings = LaBSEembeddings(so_sentences, ta_sentences)
    aligned_phrases = align( source_embeddings, target_embeddings)
    print('Number of Aligned Sentences in Document No' + str(i) + ' is :' + str(len(aligned_phrases)))
    final+=len(aligned_phrases)
    ap = []
    for key in aligned_phrases:
      l = []
      l.append(key)
      l.append(aligned_phrases[key][0])
      l.append(aligned_phrases[key][1])
      ap.append(l)
    writer = csv.writer(matching_file)
    writer.writerow(['English', 'Tamil','Distance'])
    for a in ap:
      writer.writerow(a)



LABSE embedding generation finished
Number of Aligned Sentences in Document No0 is :0
English Sentence Count for 1 Document is 716
Tamil Sentence Count for 1 Document is 743



LABSE embedding generation finished
Number of Aligned Sentences in Document No1 is :551
English Sentence Count for 2 Document is 653
Tamil Sentence Count for 2 Document is 729



LABSE embedding generation finished
Number of Aligned Sentences in Document No2 is :523
English Sentence Count for 3 Document is 666
Tamil Sentence Count for 3 Document is 744



LABSE embedding generation finished
Number of Aligned Sentences in Document No3 is :537
English Sentence Count for 4 Document is 262
Tamil Sentence Count for 4 Document is 404



LABSE embedding generation finished
Number of Aligned Sentences in Document No4 is :100
English Sentence Count for 5 Document is 796
Tamil Sentence Count for 5 Document is 895


...


LABSE embedding generation finished
Number of Aligned Sentences in Document No12 is :147
English Sentence Count for 13 Document is 1138
Tamil Sentence Count for 13 Document is 1184



LABSE embedding generation finished
Number of Aligned Sentences in Document No13 is :353
English Sentence Count for 14 Document is 1102
Tamil Sentence Count for 14 Document is 1106



LABSE embedding generation finished
Number of Aligned Sentences in Document No14 is :283
English Sentence Count for 15 Document is 1087
Tamil Sentence Count for 15 Document is 996



LABSE embedding generation finished
Number of Aligned Sentences in Document No15 is :165
English Sentence Count for 16 Document is 1285
Tamil Sentence Count for 16 Document is 1069



LABSE embedding generation finished
Number of Aligned Sentences in Document No16 is :292
